In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [2]:
BASE_URL = "https://jlptsensei.com/"

levels = {"n1": "jlpt-n1-grammar-list/", "n2": "jlpt-n2-grammar-list/", "n3": "jlpt-n3-grammar-list/", "n4": "jlpt-n4-grammar-list/", "n5": "jlpt-n4-grammar-list/"}

print("n1/n2/n3/n4/n5")
chosen_level = str(input("Choosen JLPT level: "))
print(f"{chosen_level} chosen")



n1/n2/n3/n4/n5
n3 chosen


In [3]:
html = requests.get(BASE_URL + levels[chosen_level])
soup = BeautifulSoup(html.text, "html.parser")
texts = {'grammar': [], 'en': [], 'jp': [], 'hg': []}

# num_pages = 1
pages = soup.find("p", {"class": "mb-0 mt-1"})
num_pages = int(pages.text[pages.text.index("of ")+3:pages.text.index("of")+4])
print(num_pages)
for p in range(1, num_pages+1):
    if p == 1:
        link = BASE_URL + levels[chosen_level]
    else:
        link = BASE_URL + levels[chosen_level] + f"page/{str(p)}/"
    html = requests.get(link)
    soup = BeautifulSoup(html.text, "html.parser")

    grammar_table = [[a['href'], a['title']] for a in soup.find_all('a', {"class": "jl-link jp"}, href=True) if a.text]
    print(grammar_table)
    for i in range(len(grammar_table)):
        
        html = requests.get(grammar_table[i][0])
        soup = BeautifulSoup(html.text, "html.parser")
        for a in soup.find_all("div", {"class": "example-cont py-5"}):
            texts['grammar'].append(grammar_table[i][1])
            for b in a.descendants:
                
                if b.name == 'p' and len(b['class']) > 1:
                    if b['class'][0] == 'm-0' and b['class'][1] == 'jp':
                        print(b['class'])
                        print(b.text)
                        texts['jp'].append(b.text)
                        

                elif b.name == 'div' and len(b['class']) > 1:
                    if b['class'][0] == 'alert' and b['class'][1] == 'alert-success':
                        texts['hg'].append(b.text)

                    elif b['class'][0] == 'alert' and b['class'][1] == 'alert-primary':
                        texts['en'].append(b.text)



5
[['https://jlptsensei.com/learn-japanese-grammar/%e4%b8%8a%e3%81%92%e3%82%8b-ageru-meaning/', 'Learn JLPT N3 Grammar: 上げる (ageru) Meaning'], ['https://jlptsensei.com/learn-japanese-grammar/%e3%81%82%e3%81%be%e3%82%8a-amari-meaning/', 'Learn JLPT N3 Grammar: あまり (amari) Meaning'], ['https://jlptsensei.com/learn-japanese-grammar/%e3%81%82%e3%81%be%e3%82%8a%e3%81%ab%e3%82%82-amari-ni-mo-meaning/', 'Learn JLPT N3 Grammar: あまりにも (amari ni mo) Meaning'], ['https://jlptsensei.com/learn-japanese-grammar/%e5%90%88%e3%81%86-%e3%81%82%e3%81%86-au-meaning/', 'Learn JLPT N3 Grammar: 合う (au) Meaning'], ['https://jlptsensei.com/learn-japanese-grammar/%e3%81%b0%e3%81%84%e3%81%84-ba-ii-meaning/', 'Learn JLPT N3 Grammar: ばいい (ba ii) Meaning'], ['https://jlptsensei.com/learn-japanese-grammar/%e3%81%b0%e3%82%88%e3%81%8b%e3%81%a3%e3%81%9f-ba-yokatta-meaning/', 'Learn JLPT N3 Grammar: ばよかった (ba yokatta) Meaning'], ['https://jlptsensei.com/learn-japanese-grammar/%e3%81%b0%ef%bd%9e%e3%81%bb%e3%81%a9-ba-hodo

In [75]:
print(texts['en'][20])
print(texts['jp'][20])
print(texts['hg'][20])
print(texts['grammar'][20])

print(len(texts['grammar']))

df = pd.DataFrame(columns=["grammar", "en", "jp", "hg"])
df["grammar"] = texts["grammar"]
df["en"] = texts["en"]
df["jp"] = texts["jp"]
df["hg"] = texts["hg"]

df.to_csv(f"/grammar/{chosen_level}.csv", index=False, header=False)

IndexError: list index out of range

[]
